In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/UCL Ni/RPIG/PCF-GAN-main

# !pip3 install virtualenv
# !virtualenv /content/drive/MyDrive/PCF-GAN
# !source /content/drive/MyDrive/PCF-GAN/bin/activate; pip install -r requirements.txt

!curl https://colab.chainer.org/install | sh -

import sys
sys.path.append("/content/drive/MyDrive/colab_env/lib/python3.10/site-packages")

Mounted at /content/drive
/content/drive/MyDrive/UCL Ni/RPIG/PCF-GAN-main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1553  100  1553    0     0   3360      0 --:--:-- --:--:-- --:--:--  3354
********************************************************************************
CUDA version could not be detected!
Try installing Chainer manually by: pip install chainer
********************************************************************************


In [4]:
from random import shuffle
import torch
# from src.PathCharGAN.PathCharGan import char_func_path
from src.PCFGAN.PCFGAN import char_func_path
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
import pandas as pd
from src.evaluations.test_metrics import Sig_mmd, Sig_mmd_small
from functools import partial
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# import signatory
from src.evaluations.evaluate import _train_regressor
from src.utils import to_numpy
from src.baselines.RCF_GAN import CFLossFunc

import ml_collections
import yaml
import os
import time

config_dir = "configs/" + "test_metrics.yaml"
with open(config_dir) as file:
    config = ml_collections.ConfigDict(yaml.safe_load(file))
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu_id
print(os.environ["CUDA_VISIBLE_DEVICES"])
from src.datasets.fbm_dl import FBM_data
import matplotlib.pyplot as plt

0


In [2]:
def tri_diag(tensor):  # input a 4D tensor, tri-diagonalize the last two dimensions
    # print(tensor.shape)
    for i in range(tensor.shape[0]):
        for j in range(tensor.shape[1]):
            tensor[i, j] = torch.diag(torch.diag(tensor[i, j], -1), -1) + torch.diag(torch.diag(tensor[i, j], 1), 1)
    return tensor


# def optimize_CF(
#     X_dl: torch.tensor,
#     Y_dl: torch.tensor,
#     iterations: int,
#     device,
#     M_num_samples: int = 8,
#     hidden_size: int = 8,
#     input_size: int = 2,
#     if_tri_diag: bool = False,
# ):
#     char_func = char_func_path(
#         num_samples=M_num_samples,
#         hidden_size=hidden_size,
#         input_size=input_size,
#         add_time=True,
#         init_range=1,
#     ).to(device)

#     if if_tri_diag:
#         for param in char_func.parameters():
#             param = param.detach()
#             param = tri_diag(param)
#             param.requires_grad = True

#     char_optimizer = torch.optim.Adam(char_func.parameters(), betas=(0, 0.9), lr=0.002)
#     # char_optimizer = torch.optim.SGD(char_func.parameters(), lr=0.02)
#     print("start opitmize charateristics function")
#     for i in tqdm(range(iterations)):
#         char_func.train()
#         char_optimizer.zero_grad()
#         X = next(iter(X_dl))
#         Y = next(iter(Y_dl))
#         char_loss = -char_func.distance_measure(X, Y, Lambda=0)
#         # print(char_loss)
#         # char_loss = - self.char_func.distance_measure(
#         #   self.D(x_real), self.D(x_fake))
#         char_loss.backward()
#         char_optimizer.step()

#         if if_tri_diag:
#             for param in char_func.parameters():
#                 param = param.detach()
#                 param = tri_diag(param)
#                 param.requires_grad = True

#     trained_char_func = char_func
#     return trained_char_func


class Compare_test_metrics:
    def __init__(self, X, Y, config):
        self.X = X
        self.Y = Y
        self.config = config

    def permutation_test(self, test_func, num_perm, sample_size):
        # start = time.time()
        with torch.no_grad():
            X = self.subsample(self.X, sample_size)
            Y = self.subsample(self.Y, sample_size)
            X = X.to(self.config.device)
            Y = Y.to(self.config.device)

            # print(t1)
            n, m = X.shape[0], Y.shape[0]
            combined = torch.cat([X, Y])
            H0_stats = []
            H1_stats = []

            for i in tqdm(range(num_perm)):
                idx = torch.randperm(n + m)
                H0_stats.append(
                    test_func(combined[idx[:n]], combined[idx[n:]])
                    .cpu()
                    .detach()
                    .numpy()
                )
                H1_stats.append(
                    test_func(
                        self.subsample(self.X, sample_size).to(self.config.device),
                        self.subsample(self.Y, sample_size).to(self.config.device),
                    )
                    .cpu()
                    .detach()
                    .numpy()
                )
            Q_a = np.quantile(np.array(H0_stats), q=0.95)
            Q_b = np.quantile(np.array(H1_stats), q=0.05)

            # print(statistics)
            # print(np.array(statistics))
            power = 1 - (Q_a > np.array(H1_stats)).sum() / num_perm
            type1_error = (Q_b < np.array(H0_stats)).sum() / num_perm
        # end = time.time()
        # print('time elapsed:', end - start)
        return power, type1_error, _

    def run_HT(
        self, num_run, train_X, train_Y, sample_size=200, num_permutations=500, tag=None, num_samples=64, hidden_size=4,
    ):
        model = []
        power = []
        type1_error = []
        tags = []

        train_X_dl = DataLoader(train_X.to(self.config.device), 128, shuffle=True)
        train_Y_dl = DataLoader(train_Y.to(self.config.device), 128, shuffle=True)

        initial_char_func = char_func_path(
                num_samples=num_samples,
                hidden_size=hidden_size,
                input_size=train_X.shape[-1],
                add_time=True,
                init_range=1
            ).to(self.config.device)

        untrained_power, untrained_t1error, t = self.permutation_test(
            partial(initial_char_func.distance_measure, Lambda=0),
            num_permutations,
            sample_size,
        )
        model.append("Random")
        power.append(untrained_power)
        type1_error.append(untrained_t1error)
        tags.append(tag)

        # trained_char_func = optimize_CF(
        #     train_X_dl,
        #     train_Y_dl,
        #     2000,
        #     self.config.device,
        #     num_samples,
        #     hidden_size,
        #     input_size=train_X.shape[-1],
        #     if_tri_diag=False,
        # )
        # trained_power, trained_t1error, t = self.permutation_test(
        #     partial(trained_char_func.distance_measure, Lambda=0),
        #     num_permutations,
        #     sample_size,
        # )
        # times.append(t)
        # model.append("Optimized PCF")
        # power.append(trained_power)
        # type1_error.append(trained_t1error)
        # tags.append(tag)
        # del trained_char_func

        initial_char_func_diag = char_func_path(
                num_samples=num_samples,
                hidden_size=hidden_size,
                input_size=train_X.shape[-1],
                add_time=True,
                init_range=1,
            ).to(config.device)
        for param in initial_char_func_diag.parameters():
            param.requires_grad = False
            param = tri_diag(param)
        untrained_power_diag, untrained_t1error_diag, t = self.permutation_test(
            partial(initial_char_func_diag.distance_measure, Lambda=0),
            num_permutations,
            sample_size,
        )
        model.append("Restricted Random")
        power.append(untrained_power_diag)
        type1_error.append(untrained_t1error_diag)
        tags.append(tag)

        sig_inv = char_func_path(
                num_samples=num_samples,
                hidden_size=hidden_size,
                input_size=train_X.shape[-1],
                add_time=True,
                init_range=1,
            ).to(config.device)
        for param in sig_inv.parameters():
            param.requires_grad = False
            param = param * 0
            n = 0
            for i in range(4):
                for j in range(4):
                    for k in range(4):
                        param[i, n, 0, 1] = complex(1., 0)
                        # param[i, n, 1, 0] = complex(-1., 0)
                        param[j, n, 1, 2] = complex(1., 0)
                        # param[j, n, 2, 1] = complex(-1., 0)
                        param[k, n, 2, 3] = complex(1., 0)
                        # param[k, n, 3, 2] = complex(-1., 0)
                        n += 1
        sig_inv_power, sig_inv_t1error, t = self.permutation_test(
            partial(sig_inv.distance_measure, Lambda=0),
            num_permutations,
            sample_size,
        )
        model.append("Signature Inversion")
        power.append(sig_inv_power)
        type1_error.append(sig_inv_t1error)
        tags.append(tag)

        return pd.DataFrame(
            {"model": model, "power": power, "type1 error": type1_error, "tag": tags}
        )

    def subsample(self, data, sample_size):
        idx = torch.randint(low=0, high=data.shape[0], size=[sample_size])
        return data[idx]

In [5]:
df_list = []
h_list = [0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7]

X = FBM_data(10000, dim=3, length=50, h=0.5)
train_X = FBM_data(5000, dim=3, length=50, h=0.5)

for h in h_list:
    print('h =', h)
    Y = FBM_data(10000, dim=3, length=50, h=h)
    train_Y = FBM_data(5000, dim=3, length=50, h=h)

    for _ in range(5):
        df = Compare_test_metrics(X, Y, config).run_HT(
            num_run=np.nan, train_X=train_X, train_Y=train_Y, tag=h, num_samples=64, hidden_size=4,
        )
        print(df)
        df_list.append(df)
    df = pd.concat(df_list)
    df.to_csv("numerical_results/metric_compare_fbm_" + str(h) + "_second.csv")

h = 0.3


100%|██████████| 500/500 [01:16<00:00,  6.55it/s]


                 model  power  type1 error  tag
0               Random   1.00         0.00  0.3
1    Restricted Random   0.72         0.13  0.3
2  Signature Inversion   1.00         0.00  0.3


100%|██████████| 500/500 [01:16<00:00,  6.58it/s]


                 model  power  type1 error  tag
0               Random   1.00        0.000  0.3
1    Restricted Random   0.85        0.134  0.3
2  Signature Inversion   1.00        0.000  0.3


...

                 model  power  type1 error  tag
0               Random  0.982        0.026  0.7
1    Restricted Random  0.466        0.368  0.7
2  Signature Inversion  0.966        0.042  0.7


100%|██████████| 500/500 [01:06<00:00,  7.50it/s]


                 model  power  type1 error  tag
0               Random  0.944        0.056  0.7
1    Restricted Random  0.460        0.284  0.7
2  Signature Inversion  0.956        0.036  0.7


100%|██████████| 500/500 [01:06<00:00,  7.48it/s]

                 model  power  type1 error  tag
0               Random  0.982        0.028  0.7
1    Restricted Random  0.300        0.384  0.7
2  Signature Inversion  0.974        0.036  0.7
